# **Vector DB**

## FAISS (Facebook AI Similarity Search)

##### Library to perform vector similarity (semantic) search using the ANN algorithm. (Usually used for light-weight vector search)

In [9]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)


In [10]:
# I'm gonna use this data in order to do an example with FAISS library

df = pd.read_csv('sample_data.csv')
print(df.shape)
df


(8, 2)


,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


# **Embeddings**

## SentenceTransformers🤗 

##### Framework for state-of-the-art sentence, text and image embeddings. 
##### I'm using this pre-trained model: https://huggingface.co/sentence-transformers/all-mpnet-base-v2

### 1) Creating embeddings for the text column in the dataframe

In [11]:
from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

vectors.shape # this output is (n, 768) where n is the number of sentences in the dataframe, aka, rows


/home/valenradovich/.pyenv/versions/3.10.6/envs/talklink/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 1.37MB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 116kB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 9.34MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 90.8kB/s]
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 297kB/s]
pytorch_model.bin: 100%|██████████| 438M/438M [00:40<00:00, 10.8MB/s] 
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 103kB/s]
special_tokens_map.json: 100%|██████████| 239/239 [00:00<00:00, 743kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 9.44MB/s]
tokenizer_config.json: 100%|██████████| 363/363 [00:00<00:00, 3

(8, 768)

In [12]:
dim = vectors.shape[1]
dim


768

### 2) Building FAISS Index for vectors

In [13]:
import faiss

index = faiss.IndexFlatL2(dim) # I'm using L2 distance (Euclidean distance)


### 3) Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [14]:
index.add(vectors) # I added all the vectors created by the encoder from the dataframe to the index
index


<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7f59f4dcb450> >

### 4) Encoding search text 

In [17]:
query = "I want to buy a polo t-shirt"

aux_vec = encoder.encode(query)
aux_vec.shape


(768,)

In [18]:
import numpy as np
search_vector = np.array(aux_vec).reshape(1,-1)
search_vector.shape # the index search expects a 2D array, so I reshaped the vector to (1, 768)


(1, 768)

### 5) Searching for similar vectors in the FAISS index created

In [23]:
# creating some queries to test the index

first_query = "I want to buy a polo t-shirt"
second_query = "looking for places to visit during the holidays"
third_query = "An apple a day keeps the doctor away"

search_queries = [first_query, second_query, third_query]


In [35]:
for query in search_queries:

    # doing encode and reshaping all in the same code line
    query_vector = np.array(encoder.encode(query)).reshape(1,-1)

    # I'm looking for the 2 most similar sentences to the query
    distances, positions_top_k = index.search(query_vector, k=2)

    # I'm using the first element of the array because the search returns a 2D array
    new_df = df.loc[positions_top_k[0]]


    print(f"Query: {query}")
    print(f"Top k: {positions_top_k}")
    print(f"Texts: {new_df.text.values}")
    print(f"Category: {new_df.category.values}")
    print("-" * 50)


Query: I want to buy a polo t-shirt
Top k: [[3 2]]
Texts: ['Vibrant color jeans for male are becoming a trend'
 'These are the latest fashion trends for this week']
Category: ['Fashion' 'Fashion']
--------------------------------------------------
Query: looking for places to visit during the holidays
Top k: [[6 7]]
Texts: ['Exciting vacation destinations for your next trip'
 'Maldives and Srilanka are gaining popularity in terms of low budget vacation places']
Category: ['Travel' 'Travel']
--------------------------------------------------
Query: An apple a day keeps the doctor away
Top k: [[1 0]]
Texts: ['Fruits, whole grains and vegetables helps control blood pressure'
 'Meditation and yoga can improve mental health']
Category: ['Health' 'Health']
--------------------------------------------------
